In [40]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate

chat= ChatOpenAI(temperature=0.1,
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler(),],
                 )

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character} and Your answer is in Korean.
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro" , intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final , pipeline_prompts=prompts)


chain = full_prompt | chat 

chain.invoke({
    "character" :"pirate",
    "example_question" : "what is your location?",
    "example_answer" : "ummmm That is a secret",
    "question" : "what is your location?",

})

어머나, 그건 비밀이야!

AIMessageChunk(content='어머나, 그건 비밀이야!')